In [3]:
from glob import glob
import ujson as json
import networkx as nx
import pickle
import stop_words
from dateutil import parser
from collections import defaultdict, Counter
import re
from nltk.stem import WordNetLemmatizer

filenames = glob("bitcoin/*.json")
stopwords = set(stop_words.get_stop_words('en'))
stopwords.update(['quote', 'pmquote', 'amquote', 'just', 'don', 'one', 'thing', 'even', 'way', 'maybe', 'also', 'please', 'well', 'actually', 'something',
                                         'going', 'anything', 'le', 'ever', 'say', 'see', 'likely', 'per', 'another', 'someone', 'let', 'anyone', 'doesn', 'include', 'doe', 'exactly',
                                         'january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december', 'like',
                                         'said', 'guy', 'will', 'can', 'able', 'people', 'become', 'tell', 'hey', 'much', 'many', 'lol', 'lot', 'want', 'still', 'really', 'think', 'didn',
                                         'isn', 'post', 'edited', 'share', 'facebookshare', 'twitter'])
lemmatizer = WordNetLemmatizer()

In [4]:
def parse_string(input_string):
    input_string = input_string.lower()
    input_string = re.sub(r'http\S+', ' ', input_string)
    input_string = re.sub(r'\S+.(com|org)', '', input_string)
    input_string = re.sub( "[^a-zA-Z]", " ", input_string).split()
    words = [lemmatizer.lemmatize(w) for w in input_string]
    words = [w for w in words if w not in stopwords and len(w) > 2]
    words = [w if w != 'bitcoins' else 'bitcoin' for w in words]
    return words

In [7]:
total_posts = 0
total_reple = 0
preprocessed_data = {}
preprocessed_data['user_network'] = nx.DiGraph()
preprocessed_data['time_posts'] = defaultdict(list)
preprocessed_data['user_time_posts'] = defaultdict(dict)
preprocessed_data['user_posts'] = defaultdict(list)
preprocessed_data['user_posts_num'] = defaultdict(int)
preprocessed_data['get_comment_num'] = defaultdict(int)
preprocessed_data['write_comment_num'] = defaultdict(int)
preprocessed_data['posts'] = []
voca = set()
word_freq = Counter()
for i, filename in enumerate(filenames):
    with open(filename) as f:
        data = json.load(f)
        
    for post in data:
        post_body = parse_string(post['post_body'])
        word_freq.update(post_body)
        
for i, filename in enumerate(filenames):
    print(i)
    
    with open(filename) as f:
        data = json.load(f)
        
    for post in data:        
        post_body = parse_string(post['post_body'])
        post_body = [w for w in post_body if word_freq[w] >= 10]
        if len(post_body) < 5:
            continue
        voca.update(post_body)
        post_user = post['post_user']
        posted_time = parser.parse(post['posted_time']).date()
        
        preprocessed_data['user_posts'][post_user].append(post_body)
        preprocessed_data['user_posts_num'][post_user] += 1
        preprocessed_data['time_posts'][posted_time].append(post_body)
        if posted_time in preprocessed_data['user_time_posts'][post_user]:
            preprocessed_data['user_time_posts'][post_user][posted_time].append(post_body)
        else:
            preprocessed_data['user_time_posts'][post_user][posted_time] = [post_body]
        preprocessed_data['posts'].append(post_body)
        
        for comment in post['comments']:
            comment_body = parse_string(comment['post_body'])
            comment_body = [w for w in comment_body if word_freq[w] >= 10]
            if len(comment_body) < 5:
                continue
            voca.update(comment_body)
            comment_user = comment['post_user']
            comment_time = parser.parse(comment['posted_time']).date()
                
            preprocessed_data['user_posts'][comment_user].append(comment_body)
            preprocessed_data['time_posts'][comment_time].append(comment_body)
            if posted_time in preprocessed_data['user_time_posts'][post_user]:
                preprocessed_data['user_time_posts'][post_user][posted_time].append(post_body)
            else:
                preprocessed_data['user_time_posts'][post_user][posted_time] = [post_body]
            
            preprocessed_data['posts'].append(comment_body)
            preprocessed_data['user_network'].add_edge(comment_user, post_user)
            preprocessed_data['get_comment_num'][post_user] += 1
            preprocessed_data['write_comment_num'][comment_user] += 1
            
voca = list(voca)
preprocessed_data['voca'] = voca
with open("preprocessed_bitcoin.pkl", 'wb') as f:
    pickle.dump(preprocessed_data, f)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41


In [5]:
total_posts = 0
total_reple = 0
preprocessed_data = {}
preprocessed_data['user_network'] = nx.DiGraph()
preprocessed_data['time_posts'] = defaultdict(list)
preprocessed_data['user_posts'] = defaultdict(list)
preprocessed_data['user_time_posts'] = defaultdict(dict)
preprocessed_data['posts'] = []
voca = set()
word_freq = Counter()
    
with open("etherium_forum.json") as f:
    data = json.load(f)

for post in data:
    post_body = parse_string(post['post_body'])
    word_freq.update(post_body)

for post in data:
    post_body = parse_string(post['post_body'])
    post_body = [w for w in post_body if word_freq[w] >= 10]
    if len(post_body) < 5:
        continue
    voca.update(post_body)
    post_user = post['post_user']
    posted_time = parser.parse(post['posted_time']).date()
    
    preprocessed_data['user_posts'][post_user].append(post_body)
    preprocessed_data['time_posts'][posted_time].append(post_body)
    if posted_time in preprocessed_data['user_time_posts'][post_user]:
            preprocessed_data['user_time_posts'][post_user][posted_time].append(post_body)
    else:
        preprocessed_data['user_time_posts'][post_user][posted_time] = [post_body]
    preprocessed_data['posts'].append(post_body)

    for comment in post['comments']:
        comment_body = parse_string(comment['post_body'])
        comment_body = [w for w in comment_body if word_freq[w] >= 10]
        if len(comment_body) < 5:
            continue
        voca.update(comment_body)
        comment_user = comment['post_user']
        comment_time = parser.parse(comment['posted_time']).date()
        
        preprocessed_data['user_posts'][comment_user].append(comment_body)
        preprocessed_data['time_posts'][comment_time].append(comment_body)
        if posted_time in preprocessed_data['user_time_posts'][post_user]:
            preprocessed_data['user_time_posts'][post_user][posted_time].append(post_body)
        else:
            preprocessed_data['user_time_posts'][post_user][posted_time] = [post_body]
        
        preprocessed_data['posts'].append(comment_body)
        preprocessed_data['user_network'].add_edge(comment_user, post_user)
            
voca = list(voca)
preprocessed_data['voca'] = voca
with open("preprocessed_etherium.pkl", 'wb') as f:
    pickle.dump(preprocessed_data, f)

In [4]:
total_posts = 0
total_reple = 0
preprocessed_data = {}
preprocessed_data['user_network'] = nx.DiGraph()
preprocessed_data['time_posts'] = defaultdict(list)
preprocessed_data['user_posts'] = defaultdict(list)
preprocessed_data['user_time_posts'] = defaultdict(dict)
preprocessed_data['posts'] = []
voca = set()
word_freq = Counter()
    
with open("ripple_forum.json") as f:
    data = json.load(f)

for post in data:
    post_body = parse_string(post['post_body'])
    word_freq.update(post_body)

for post in data:
    post_body = parse_string(post['post_body'])
    post_body = [w for w in post_body if word_freq[w] >= 10]
    if len(post_body) < 5:
        continue
    voca.update(post_body)
    post_user = post['post_user']
    posted_time = parser.parse(post['posted_time']).date()
    
    preprocessed_data['user_posts'][post_user].append(post_body)
    preprocessed_data['time_posts'][posted_time].append(post_body)
    if posted_time in preprocessed_data['user_time_posts'][post_user]:
            preprocessed_data['user_time_posts'][post_user][posted_time].append(post_body)
    else:
        preprocessed_data['user_time_posts'][post_user][posted_time] = [post_body]
    preprocessed_data['posts'].append(post_body)

    for comment in post['comments']:
        comment_body = parse_string(comment['post_body'])
        comment_body = [w for w in comment_body if word_freq[w] >= 10]
        if len(comment_body) < 5:
            continue
        voca.update(comment_body)
        comment_user = comment['post_user']
        comment_time = parser.parse(comment['posted_time']).date()
        
        preprocessed_data['user_posts'][comment_user].append(comment_body)
        preprocessed_data['time_posts'][comment_time].append(comment_body)
        if posted_time in preprocessed_data['user_time_posts'][posted_time]:
            preprocessed_data['user_time_posts'][posted_time][post_user].append(post_body)
        else:
            preprocessed_data['user_time_posts'][posted_time][post_user] = [post_body]
        
        preprocessed_data['posts'].append(comment_body)
        preprocessed_data['user_network'].add_edge(comment_user, post_user)
            
voca = list(voca)
preprocessed_data['voca'] = voca
with open("preprocessed_ripple.pkl", 'wb') as f:
    pickle.dump(preprocessed_data, f)